In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

#df = pd.read_parquet('Downloads/airbnb_parquets')
#df.to_csv('csv_printout.csv')

In [2]:
def append_fig_to_html(list_of_figs):
    for fig in list_of_figs:
        with open("reports/report_draft.html",'a') as f:
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

# write this later
"""
def overwrite_html(list_of_figs):
    
    with open("reports/report_draft.html",'w') as f:
    f.write(listing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_v_numlist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
"""

def calculate_mortgage(home_value, interest_rate, num_years, down_payment_pct=0):
    per_payment_interest = 0
    loan_value = 0

    if down_payment_pct >= 1:
        down_payment = down_payment_pct/100 * home_value
        loan_value = home_value - down_payment
    else:
        down_payment = down_payment_pct * home_value
        loan_value = home_value - down_payment

    if loan_value/home_value < 0.80: 
        # insert pmi calc here
        pmi = 0.01  # using near average value here
        pmi_cost = 0.0007*home_value    # shot in the dark after interpolating nerdwallet calculator
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1) + pmi_cost

        mortgage = np.round(mortgage, 2)
        return mortgage
    else:
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1)

        mortgage = np.round(mortgage, 2)
        return mortgage

def calculate_roi(airbnb_daily_price, occupancy_rate, monthly_mortgage, monthly_maintenence=0, monthly_taxes=0):
    gross_rev = airbnb_daily_price * occupancy_rate * 365/12
    net_rev = gross_rev - monthly_maintenence - monthly_taxes
    profit = net_rev - monthly_mortgage
    roi = profit/monthly_mortgage
    roi = roi
    return roi

def set_location_desc(df,lat,long,geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Lat casted at value at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Long casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))

    state = address.get('state', '')
    print("---Get state value at %s seconds ---" % (time.time() - start_time))

    country = address.get('country')
    print("---Get country value at %s seconds ---" % (time.time() - start_time))
    country_code = address.get('country_code')
    print("---Get country_code value at %s seconds ---" % (time.time() - start_time))
    return city

def set_city(lat, long, geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Latitude casted to string at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Longitude casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location json from geopy at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address json at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))
    return city

def set_state(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    state = address.get('state', '')
    return state

def set_country(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    country = address.get('country')
    return country

def set_country_code(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(lat+","+long)
    address = location.raw['address']
    country_code = address.get('country_code')
    return country_code

def set_zipcode(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    zipcode = address.get('postcode', '')
    return zipcode


In [4]:
# Load in listing data
nc_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/north_carolina'
first_file = ''
for file in os.listdir(nc_dir):
    first_file = os.path.join(nc_dir,file)
    break
listing_data = pd.read_parquet(first_file)
print(listing_data.shape)
for file in os.listdir(nc_dir):
    next_file = os.path.join(nc_dir,file)
    if next_file != first_file:
        next_listing = pd.read_parquet(next_file)
        listing_data = listing_data.append(next_listing)
print(listing_data.shape)

ne_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/vt_nh'
for file in os.listdir(ne_dir):
    next_file = os.path.join(ne_dir,file)
    next_listing = pd.read_parquet(next_file)
    listing_data = listing_data.append(next_listing)
print(listing_data.shape)

# Load in occupancy data
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')

# Load in price data
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

(20, 25)
(19427, 25)
(35284, 25)


In [5]:
# Clean up listing_data
conditions = [
    listing_data['baths'].str.contains('shared', na=False),
    listing_data['baths'].str.contains('Shared', na=False)
]

values = ['Shared', 'Shared']

listing_data['baths_type'] = np.select(conditions, values, default='Private')

conditions_loc = [
    listing_data['top_lat'] > 35.5
]

values_loc = ['New England']

listing_data['Region'] = np.select(conditions_loc, values_loc, default='Carolinas')


conditions_halfbath = [
    listing_data['baths'].str.contains('Half-bath', na=False),
    listing_data['baths'].str.contains('Shared half-bath', na=False),
    listing_data['baths'].str.contains('Private half-bath', na=False)
]

values_halfbath = [0.5,0.5,0.5]

listing_data['baths'] = np.select(conditions_halfbath, values_halfbath, default=listing_data['baths'])

listing_data['baths_no'] = listing_data['baths'].str.split(' ').str[0]
listing_data['beds_no'] = listing_data['beds'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['title'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['guest_no'].astype('float')
listing_data['id'] = listing_data['id'].astype('float64')
listing_data['id'] = listing_data['id'].astype('str')
listing_data['town'] = listing_data['town'].str.lower()
listing_data = listing_data[listing_data.lat.isna() == False]
#listing_data['city_geopy'] = listing_data.apply(lambda row: set_city(row['lat'], row['lng']), axis=1)
#listing_data['state_geopy'] = listing_data.apply(lambda row: set_state(row['lat'], row['lng']), axis=1)
#listing_data['country_geopy'] = listing_data.apply(lambda row: set_country(row['lat'], row['lng']), axis=1)
#listing_data['country_code_geopy'] = listing_data.apply(lambda row: set_country_code(row['lat'], row['lng']), axis=1)
#listing_data['zipcode_geopy'] = listing_data.apply(lambda row: set_zipcode(row['lat'], row['lng']), axis=1)

# Clean up occ_data
#occ_data['id'] = occ_data['id'].astype('int')
occ_data['id'] = occ_data['id'].astype('str')

# Clean up price_data
#price_data['id'] = price_data['id'].astype('int')
price_data['id'] = price_data['id'].astype('str')

In [6]:
city_data = pd.read_csv('resource_data/uscities.csv')
home_value = pd.read_csv('resource_data/home_value_zillow.csv')

In [28]:
combined_data.columns

Index(['index', 'id', 'name', 'price', 'displayPrice', 'monthlyPriceFactor',
       'weeklyPriceFactor', 'avgRating', 'reviewsCount', 'isNewListing',
       'isSuperhost', 'lat', 'lng', 'personCapacity', 'size', 'beds', 'baths',
       'title', 'property_type', 'town', 'top_lat', 'right_long', 'bottom_lat',
       'left_long', 'url', 'baths_type', 'Region', 'baths_no', 'beds_no',
       'guest_no', 'occupancy_rate', 'median_cleaning_fee',
       'median_service_fee'],
      dtype='object')

In [37]:
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
combined_data = listing_data.merge(occ_rate, on = 'id')
combined_data.rename(columns = {'available':'occupancy_rate'}, inplace = True)

cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
combined_data = combined_data.merge(cleaning_fee, on='id')
combined_data = combined_data.merge(service_fee, on='id')
combined_data['median_total_price'] = combined_data['price'] + combined_data['median_cleaning_fee'] + combined_data['median_service_fee']

home_value['RegionName'] = home_value['RegionName'].str.lower()
home_value_reduced = home_value[['RegionName','CountyName', 'Metro', 'State', '2022-04-30']]
combined_data = combined_data.merge(home_value, left_on='town', right_on='RegionName')


In [38]:
# Adding avg mortgage and median ROI
combined_data['avg_30_yr_mort'] = combined_data.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)
combined_data['median_ROI'] = combined_data.apply(lambda row: calculate_roi(row['median_total_price'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)

Index(['Region', 'property_type', 'median_ROI'], dtype='object')

In [ ]:
lists_per_city = listing_data.groupby(['town'])['id'].count().reset_index()
lists_per_city.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
median_price_per_city = listing_data.groupby(['town'])['price'].median().reset_index()
median_price_per_city.rename(columns = {'price':'median_price_per_city'}, inplace = True)

# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')


# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
#total_price = price_data.groupby(['id'])['total_price'].median().reset_index()
#total_price.rename(columns = {'total_price':'median_total_price'}, inplace = True)

listing_occ_rate_fees = listing_occ_rate.merge(cleaning_fee, on='id')
listing_occ_rate_fees = listing_occ_rate_fees.merge(service_fee, on='id')
#listing_occ_rate_fees = listing_occ_rate_fees.merge(total_price, on='id')

median_occ_rate_per_city = listing_occ_rate_fees.groupby(['town'])['available'].median().reset_index()
median_occ_rate_per_city.rename(columns = {'available':'occupancy_rate'}, inplace = True)

median_cleaning_fee_per_city = listing_occ_rate_fees.groupby(['town'])['median_cleaning_fee'].median().reset_index()
median_service_fee_per_city = listing_occ_rate_fees.groupby(['town'])['median_service_fee'].median().reset_index()
#median_total_price_per_city = listing_occ_rate_fees.groupby(['town'])['median_total_price'].median().reset_index()
#fees_and_price = cleaning_fee.merge(service_fee, on = 'id')
#fees_and_price = fees_and_price.merge(total_price, on = 'id')

lists_town_price = lists_per_city.merge(median_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price = lists_town_price.merge(median_cleaning_fee_per_city, left_on = 'town', right_on = 'town')
lists_town_price = lists_town_price.merge(median_service_fee_per_city, left_on = 'town', right_on = 'town')
#lists_town_price = lists_town_price.merge(median_total_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price['median_total_price'] = lists_town_price['median_price_per_city'] + lists_town_price['median_cleaning_fee'] + lists_town_price['median_service_fee']
lists_town_price_occ = lists_town_price.merge(median_occ_rate_per_city, left_on = 'town', right_on = 'town')
lists_town_home_price_occ = lists_town_price_occ.merge(city_home, left_on = 'town', right_on = 'city')

lists_town_home_price_occ_vt = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'VT')]
lists_town_home_price_occ_nh = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NH')]
lists_town_home_price_occ_nc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NC')]
lists_town_home_price_occ_sc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'SC')]

lists_town_home_price_occ_all = lists_town_home_price_occ_vt.append(lists_town_home_price_occ_nh)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_nc)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_sc)

lists_town_home_price_occ_all['avg_30_yr_mort'] = lists_town_home_price_occ_all.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)
lists_town_home_price_occ_all['median_ROI'] = lists_town_home_price_occ_all.apply(lambda row: calculate_roi(row['median_price_per_city'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)

In [ ]:
# grouping by town and guests

city_home['city'] = city_home['city'].str.lower()
#print("city home shape: " + str(city_home.shape))
lists_per_city_guest = listing_data.groupby(['town', 'guest_no'])['id'].count().reset_index()
lists_per_city_guest.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
#print("lists_per_city_guest shape: " + str(lists_per_city_guest.shape))
median_price_per_city_guest = listing_data.groupby(['town', 'guest_no'])['price'].median().reset_index()
median_price_per_city_guest.rename(columns = {'price':'median_price_per_city'}, inplace = True)
#print("median_price_per_city_guest shape: " + str(median_price_per_city_guest.shape))


# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')
#print("listing_occ_rate shape: " + str(listing_occ_rate.shape))



# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
cleaning_fee.rename(columns = {'cleaning_fee':'median_cleaning_fee'}, inplace = True)
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
service_fee.rename(columns = {'service_fee':'median_service_fee'}, inplace = True)
#total_price = price_data.groupby(['id'])['total_price'].median().reset_index()
#total_price.rename(columns = {'total_price':'median_total_price'}, inplace = True)

listing_occ_rate_fees = listing_occ_rate.merge(cleaning_fee, on='id')
#print("listing_occ_rate_fees shape: " + str(listing_occ_rate_fees.shape))
listing_occ_rate_fees = listing_occ_rate_fees.merge(service_fee, on='id')
#print("listing_occ_rate_fees shape: " + str(listing_occ_rate_fees.shape))

#listing_occ_rate_fees = listing_occ_rate_fees.merge(total_price, on='id')

median_occ_rate_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['available'].median().reset_index()
median_occ_rate_per_city_guest.rename(columns = {'available':'occupancy_rate'}, inplace = True)
#print("median_occ_rate_per_city_guest shape: " + str(median_occ_rate_per_city_guest.shape))


median_cleaning_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_cleaning_fee'].median().reset_index()
#print("median_occ_rate_per_city_guest shape: " + str(median_occ_rate_per_city_guest.shape))

median_service_fee_per_city_guest = listing_occ_rate_fees.groupby(['town', 'guest_no'])['median_service_fee'].median().reset_index()
#print("median_service_fee_per_city_guest shape: " + str(median_service_fee_per_city_guest.shape))


lists_town_price_guest = lists_per_city_guest.merge(median_price_per_city_guest, on = ['town', 'guest_no'])
#print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_guest = lists_town_price_guest.merge(median_cleaning_fee_per_city_guest, on = ['town', 'guest_no'])
#print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_guest = lists_town_price_guest.merge(median_service_fee_per_city_guest, on = ['town', 'guest_no'])
#print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

#lists_town_price = lists_town_price.merge(median_total_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price_guest['median_total_price'] = lists_town_price_guest['median_price_per_city'] + lists_town_price_guest['median_cleaning_fee'] + lists_town_price_guest['median_service_fee']
#print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

lists_town_price_occ_guest = lists_town_price_guest.merge(median_occ_rate_per_city_guest, on = ['town', 'guest_no'])
#print("lists_town_price_guest shape: " + str(lists_town_price_guest.shape))

city_home_needed_cols = city_home[['city','state_id','population','2022-04-30']]
#print("city_home_needed_cols shape: " + str(city_home_needed_cols.shape))

lists_town_home_price_occ_guest = lists_town_price_occ_guest.merge(city_home_needed_cols, how='inner', left_on = 'town', right_on = 'city')
#print("lists_town_home_price_occ_guest shape: " + str(lists_town_home_price_occ_guest.shape))

# Calculate 30 year mortgage for town
lists_town_home_price_occ_guest['avg_30_yr_mort'] = lists_town_home_price_occ_guest.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)
# Calculate median ROI for town
lists_town_home_price_occ_guest['median_ROI'] = lists_town_home_price_occ_guest.apply(lambda row: calculate_roi(row['median_price_per_city'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)


wb_data = lists_town_home_price_occ_guest[lists_town_home_price_occ_guest['town'] == 'wrightsville beach']
#print("wb_data shape: " + str(wb_data.shape))

# Create pivot table for nc towns
lists_town_home_price_occ_guest_nc = lists_town_home_price_occ_guest[(lists_town_home_price_occ_guest['state_id'] == 'NC')]
num_lists_pivot = lists_town_home_price_occ_guest_nc.pivot(index='town', columns='guest_no', values='num_lists_in_town')